In [1]:
import pickle
import nltk

import gensim

In [2]:
import os
os.chdir("prepared_corpora/opinosis/")


In [15]:
phrases = None
with open("phrases.txt", "r") as phrases_fh:
    phrases = list(phrases_fh)

#tokenised_phrases = nltk.tokenize.SpaceTokenizer().tokenize_sents(map(str.lower, phrases))\
tokenised_phrases = nltk.tokenize.TreebankWordTokenizer().tokenize_sents(map(str.lower, phrases))

with open("tokenized_phrases.txt","w") as fh:
    for phrase in tokenised_phrases:
        fh.write(" ".join(phrase))
        fh.write("\n")
    

In [9]:
def infer_and_save(model,tokenised_phrases, savename):
    with open(savename, "w") as outvectors_fh:
        for tockenised_line in tokenised_phrases:
            vector = model.infer_vector(tockenised_line)
            outvectors_fh.write(str(vector.tolist())[1:-1])
            outvectors_fh.write("\n")

In [11]:
model = gensim.models.doc2vec.Doc2Vec.load("../../models/wiki_sentence_model_concat_model")
infer_and_save(model, tokenised_phrases,"outVectors_wiki_sentence_concat_pvdm.csv")

In [12]:
model = gensim.models.doc2vec.Doc2Vec.load("../../models/wiki_sentence_model_dbow_model")
infer_and_save(model, tokenised_phrases,"outVectors_wiki_sentence_dbow.csv")

In [44]:
import numpy as np
import gensim.corpora.dictionary as gensim_bow


class BowModel(object):
    def __init__(self, tokenised_phrases):
        self.bow_dict = gensim_bow.Dictionary(tokenised_phrases)
        
    def _unsparse(self, bow_kv_list):
        ret = np.zeros(len(self.bow_dict))
        for key, value in bow_kv_list:
            ret[key]=value
        return ret
    
    def infer_vector(self,tokenised_phrase):
        bow = self.bow_dict.doc2bow(tokenised_phrase)
        return self._unsparse(bow)
    
        

In [48]:
bow_model = BowModel(tokenised_phrases)
infer_and_save(bow_model, tokenised_phrases,"outVectors_bow.csv")

#Word Embeeding Models

In [ ]:

#Loads googles word2vec_embeddings
function load_word2vec_embeddings(embedding_file, max_stored_vocab_size = 1000000)
    #Note: I know there actually <10^6 words in the vocab, when phrases are exlusded, so lock the vocab size to this to save 70%RAM
    #Words are loosely organised by commonness,  AFAICT
    fh = open(embedding_file,"r")
    vocab_size, vector_size = @pipe readline(fh)|> split |> map(int, _)
    max_stored_vocab_size = min(max_stored_vocab_size, vocab_size) #if using a small vocab then there is a chance you might be willing ot store more words than it has
    
    
    indexed_words = Array(String,max_stored_vocab_size)
    word_indexes = Dict{String,Int64}()
    LL = Array(Float32,(vector_size, max_stored_vocab_size))

    #Add a Zero vector for the unknown words
    LL[:,1]*=0
    indexed_words[1]=UNKNOWN_WORD
    word_indexes[UNKNOWN_WORD]=1

    #Add all from data
    index = 2
    for _ in 1:vocab_size
        word = readuntil(fh,' ') |> strip #Technically this is 'ISO-8859-1' may have to deal with encoding issues
        vector = read(fh, Float32,vector_size ) 

        if !contains(word, "_") #If it isn't a phrase
            LL[:,index]=vector./norm(vector)
            indexed_words[index] = word
            word_indexes[word] = index
            
            index+=1
            if index>max_stored_vocab_size
                warn("Max Vocab size exceeded. More words are available if you want.")
                break
            end
        end
        
    end
    close(fh)
    
    LL = LL[:,1:index-1] #throw away unused columns
    indexed_words = indexed_words[1:index-1] #throw away unused columns
    LL, word_indexes, indexed_words
end

In [50]:
import numpy as np

In [237]:
import codecs
import io

def read_until(until_char,stream):
    """Consumes up to and including the next occurance of the until_char. Raises EOFError if reached end of file without finign the untilchar"""
    ret = b''
    while(True):
        read_char=stream.read(1)
        if len(read_char)==0:
            raise EOFError()
        elif read_char==until_char:
            return ret
        else:
            ret+=read_char
        
        
def load_word2vec_embeddings(embedding_file, words_to_keep=set()):
    """
    words_to_keep: if this is a nonempty set then only those words will be kept. if it is empty (or not given) then all words will be kept
    """
    
    word_vectors = dict()
    
    with io.open(embedding_file,"rb") as fh:
        #return fh
        vocab_size, vector_size = [int(s) for s in fh.readline().split()]
        for ii in range(vocab_size):
            word = read_until(b' ', fh)
            word = word.decode(encoding='iso-8859-1')
            
            #print(word)
            encoded_embedding = fh.read(4*vector_size)#4 bytes for Float32
            
            if len(words_to_keep)==0 or word in words_to_keep:
                word_vectors[word] = np.fromstring(encoded_embedding, dtype=np.float32, count=vector_size)    
    
    return word_vectors
                
            
        
        
        

In [238]:
word_vectors = load_word2vec_embeddings("../../../../../Resources/example_code/word2vec/GoogleNews-vectors-negative300.bin", bow_model.bow_dict.token2id.keys())

In [244]:
set(bow_model.bow_dict.token2id.keys()).difference(word_vectors.keys())

{'!',
 "'",
 "'manager",
 "'s",
 "'select",
 "'tasting",
 ',',
 '.',
 '10',
 '100',
 '101',
 '15',
 '150',
 '20',
 '21',
 '22',
 '30',
 '39',
 "5'10",
 '75',
 '79',
 '90',
 '92',
 '?',
 'a',
 'acqure',
 'and',
 'can\x92t',
 'celan',
 'courtious',
 'e-ink',
 'earily',
 'favourable',
 'fishermans',
 'fisherman\x92s',
 'gloucester',
 'inmaculate',
 'jekyll',
 'kensington',
 'latei',
 'of',
 'offree',
 'openzone',
 'positivesthe',
 'relaxment',
 'socialise',
 'to',
 'toyotas',
 'travellers',
 'unaccomodating',
 'visata',
 'w7',
 'witheach',
 '~10'}

In [247]:
class MeanOfWordEmbeedingsModel(object):
    def __init__(self, tokenised_phrases, word_embeddings):
        self.word_vectors = word_embeddings

    
    def infer_vector(self,tokenised_phrase):
        n_words = 0
        sum_of_word_embeddings = np.zeros(300)
        for word in tokenised_phrase:
            if word in self.word_vectors:
                n_words+=1
                sum_of_word_embeddings+=self.word_vectors[word]
        return sum_of_word_embeddings/n_words


In [248]:
model = MeanOfWordEmbeedingsModel(tokenised_phrases, word_vectors)
infer_and_save(model, tokenised_phrases,"outVectors_mowe.csv")

In [325]:
class HistogramOfWordEmbeedingsModel(object):
    def __init__(self, tokenised_phrases, word_embeddings, n_bins = 10):
        self.word_vectors = word_embeddings
        min_value = np.row_stack(word_vectors.values()).min()
        max_value = np.row_stack(word_vectors.values()).max()
        self.bins = np.linspace(min_value,max_value,n_bins)
        
    
    def infer_vector(self,tokenised_phrase):
        word_embeddings = np.row_stack([self.word_vectors[word] for word in tokenised_phrase if word in self.word_vectors])
        def get_hists(col_slice):
            return np.histogram(col_slice, bins=self.bins)[0]
            
        return np.apply_along_axis(get_hists,0,word_embeddings).flatten()


In [326]:
model = HistogramOfWordEmbeedingsModel(tokenised_phrases, word_vectors)
infer_and_save(model, tokenised_phrases,"outVectors_howe.csv")

In [328]:
len(tokenised_phrases)

669